In [9]:
from numpy import *
import pandas as pd
import time


def loadDataSet(pwd):
    data = pd.read_excel(pwd)
    df = data.drop('学生ID',axis=1, inplace=False)
    x = df.values.tolist()
    fre_list = []

    for i in x:
        sign = 1
        temp = []
        for j in i:
            if sign == 13:
                sign += 4
            if j == 1:
                temp.append(sign)
            sign += 1
        fre_list.append(temp)
        
    return fre_list

In [10]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset,C1))

In [11]:
 
def scanD(D,Ck,minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            # retList.insert(0,key)
            retList.append(key)
        supportData[key] = support
    return retList, supportData

In [12]:
def aprioriGen(Lk, k):
    lenLk = len(Lk)
    temp_dict = {}
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = Lk[i]|Lk[j]
            if len(L1) == k:
                if not L1 in temp_dict:
                    temp_dict[L1] = 1
    return list(temp_dict)

In [13]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    # print("C1",C1)
    D = list(map(set,dataSet))
    L1,supportData = scanD(D, C1, minSupport)
    # print("L1",L1)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2],k) 
        Lk, supK = scanD(D,Ck,minSupport) 
        supportData.update(supK) 
        L.append(Lk)
        k += 1
    return L,supportData

In [14]:
def calcConf(freqSet, H, supportData, br1, minConf = 0.7): 
    prunedH = []
    for conseq in H:  # conseq = right key，freqSet = 原始频繁项, freqSet-conseq = left key
        conf = supportData[freqSet]/supportData[freqSet-conseq] 
        if conf>= minConf:
            print(freqSet-conseq,"-->",conseq,"\tconf:",conf)
            br1.append((freqSet-conseq,conseq,conf))
        else:
            prunedH.append(conseq)
    return prunedH


In [15]:
def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7): 
    is_find = True  
    m = 1 
    Hmp1 = H   
    while is_find:
        if len(freqSet) > m: 
            if m > 1: 
                Hmp1 = aprioriGen(H,m) 
            H_no = calcConf(freqSet, Hmp1, supportData, br1, minConf) 
            if len(H_no) != 0: 
                H_no = list(set(frozenset([item]) for row in Hmp1 for item in row))
                H = list(set(H)-set(H_no))  
            m = m + 1 
            if len(H) < m:  
                is_find = False
        else:  
            is_find = False

In [16]:
def generateRules(L, supportData, minConf=0.7):  
    bigRuleList = []
    for i in range(1, len(L)): 
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i>1: 
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:  
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    return bigRuleList

In [17]:
if __name__ == "__main__":
    dataSet = loadDataSet('./数据集离散化.xlsx')
    begin_time = time.time()
    L,suppData = apriori(dataSet)
 
    print("--------------------------Total results----------------------------")
    rules = generateRules(L, suppData, minConf=0.5)
    print("--------------------------Final results----------------------------")
    for i in rules:
        if i[2]>=0.96:
            print(i)
    end_time = time.time()
    print("-----------------------------Time----------------------------------")
    print("run {} seconds".format(end_time-begin_time))

--------------------------Total results----------------------------
frozenset({2}) --> frozenset({1}) 	conf: 0.9432246103933722
frozenset({1}) --> frozenset({2}) 	conf: 0.8493400803380458
frozenset({3}) --> frozenset({1}) 	conf: 0.9381493506493507
frozenset({1}) --> frozenset({3}) 	conf: 0.9044290260316136
frozenset({4}) --> frozenset({1}) 	conf: 0.936549250968824
frozenset({1}) --> frozenset({4}) 	conf: 0.844697167301372
frozenset({5}) --> frozenset({1}) 	conf: 0.9422032849975945
frozenset({1}) --> frozenset({5}) 	conf: 0.7152172778966038
frozenset({6}) --> frozenset({1}) 	conf: 0.938552713661884
frozenset({1}) --> frozenset({6}) 	conf: 0.941833168136053
frozenset({1}) --> frozenset({8}) 	conf: 0.6050915540716783
frozenset({8}) --> frozenset({1}) 	conf: 0.9416301347621368
frozenset({10}) --> frozenset({1}) 	conf: 0.9376347951114307
frozenset({1}) --> frozenset({10}) 	conf: 0.5443163440972403
frozenset({11}) --> frozenset({1}) 	conf: 0.9386569270357765
frozenset({1}) --> frozenset({11}

frozenset({2, 4, 5}) --> frozenset({1, 3}) 	conf: 0.8853687617101005
frozenset({2, 3, 5}) --> frozenset({1, 4}) 	conf: 0.8391992250565063
frozenset({2, 3, 4}) --> frozenset({1, 5}) 	conf: 0.7464101091326822
frozenset({1, 4, 5}) --> frozenset({2, 3}) 	conf: 0.866766716691679
frozenset({1, 3, 5}) --> frozenset({2, 4}) 	conf: 0.8172313497366559
frozenset({1, 3, 4}) --> frozenset({2, 5}) 	conf: 0.6995962314939435
frozenset({1, 2, 5}) --> frozenset({3, 4}) 	conf: 0.8283005338220061
frozenset({1, 2, 4}) --> frozenset({3, 5}) 	conf: 0.7384571672112515
frozenset({1, 2, 3}) --> frozenset({4, 5}) 	conf: 0.6903512849458795
frozenset({2, 3, 4, 6}) --> frozenset({1}) 	conf: 0.9432993526304041
frozenset({1, 3, 4, 6}) --> frozenset({2}) 	conf: 0.8905514576747454
frozenset({1, 2, 4, 6}) --> frozenset({3}) 	conf: 0.9361246492394034
frozenset({1, 2, 3, 6}) --> frozenset({4}) 	conf: 0.8749396093588239
frozenset({1, 2, 3, 4}) --> frozenset({6}) 	conf: 0.9653518123667377
frozenset({3, 4, 6}) --> frozenset(